<a href="https://colab.research.google.com/github/unverciftci/agents_intro/blob/main/N4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
# Tutorial 4: AI that Takes Actions
# Goal: Agent that performs real-world actions

import re
import os
from datetime import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load AI (same as previous tutorials)
print("Loading AI...")
model_name = "Qwen/Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
print("✅ AI ready!")

# Step 1: Memory system (from Tutorial 3)
class AgentMemory:
    """Memory system with action logging"""

    def __init__(self):
        self.conversation_history = []
        self.learned_facts = {}
        self.completed_actions = []

    def add_conversation(self, user_message, agent_response):
        self.conversation_history.append({
            'timestamp': datetime.now().strftime("%H:%M"),
            'user': user_message,
            'agent': agent_response
        })
        if len(self.conversation_history) > 6:
            self.conversation_history = self.conversation_history[-6:]

    def learn_fact(self, key, value):
        self.learned_facts[key] = value
        print(f"🧠 Learned: {key} = {value}")

    def log_action(self, action, result):
        """Remember completed actions"""
        self.completed_actions.append({
            'timestamp': datetime.now().strftime("%H:%M"),
            'action': action,
            'result': result
        })
        if len(self.completed_actions) > 8:
            self.completed_actions = self.completed_actions[-8:]

    def get_context(self):
        context = ""

        if self.learned_facts:
            context += "What I know:\n"
            for key, value in self.learned_facts.items():
                context += f"- {key}: {value}\n"
            context += "\n"

        if self.completed_actions:
            context += "Recent actions I took:\n"
            for action in self.completed_actions[-2:]:
                context += f"[{action['timestamp']}] {action['action']}\n"
            context += "\n"

        if self.conversation_history:
            context += "Recent conversation:\n"
            for conv in self.conversation_history[-2:]:
                context += f"You: {conv['user']}\nMe: {conv['agent']}\n"
            context += "\n"

        return context

memory = AgentMemory()
print("✅ Memory system ready")

# Step 2: Action tools (real world!)
def create_file(filename_and_content):
    """Create a real file"""
    try:
        if ": " in filename_and_content:
            filename, content = filename_and_content.split(": ", 1)
            with open(filename.strip(), 'w') as f:
                f.write(content.strip())
            return f"✅ Created file: {filename.strip()}"
        else:
            return "❌ Format: filename.txt: content here"
    except Exception as e:
        return f"❌ Error: {e}"

def read_file(filename):
    """Read a real file"""
    try:
        with open(filename.strip(), 'r') as f:
            content = f.read()
        return f"📄 Content of {filename}:\n{content}"
    except FileNotFoundError:
        return f"❌ File {filename} not found"
    except Exception as e:
        return f"❌ Error: {e}"

def list_files():
    """List files in current directory"""
    try:
        files = [f for f in os.listdir('.') if os.path.isfile(f)]
        return f"📁 Files:\n" + "\n".join(f"  • {f}" for f in files)
    except Exception as e:
        return f"❌ Error: {e}"

def set_reminder(reminder_text):
    """Create a reminder by writing to file"""
    try:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")
        reminder = f"[{timestamp}] REMINDER: {reminder_text}\n"

        with open("reminders.txt", "a") as f:
            f.write(reminder)

        return f"⏰ Reminder set: {reminder_text}"
    except Exception as e:
        return f"❌ Error: {e}"

def send_email_simulation(email_details):
    """Simulate sending email (demo only)"""
    try:
        if ": " in email_details:
            parts = email_details.split(": ", 2)
            if len(parts) >= 3:
                email, subject, body = parts
                return f"📧 Email sent!\n  To: {email}\n  Subject: {subject}\n  Body: {body}"
        return "❌ Format: email@domain.com: Subject: Message body"
    except Exception as e:
        return f"❌ Error: {e}"

# Action tools registry
ACTION_TOOLS = {
    "create_file": create_file,
    "read_file": read_file,
    "list_files": list_files,
    "set_reminder": set_reminder,
    "send_email": send_email_simulation,
    "remember": lambda fact: memory.learn_fact("note", fact) or f"Remembered: {fact}"
}

print("✅ Action tools ready: create_file, read_file, list_files, set_reminder, send_email")

# Step 3: AI function
def ask_ai(prompt):
    """Send prompt to AI"""
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True, enable_thinking=False
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=140,
            temperature=0.3,
            do_sample=True,
        )

    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    return tokenizer.decode(output_ids, skip_special_tokens=True).strip()

# Step 4: Action agent
def action_agent(user_message):
    """Agent that takes real actions"""

    context = memory.get_context()

    system_prompt = f"""You are an AI assistant that can take real actions.

{context}

Available actions:
- TOOL:create_file:filename.txt: content here
- TOOL:read_file:filename.txt
- TOOL:list_files: (no parameters)
- TOOL:set_reminder:reminder text
- TOOL:send_email:user@email.com: Subject: Message body
- TOOL:remember:fact to remember

Examples:
User: Create a file called notes.txt with my ideas
AI: I'll create that file for you! TOOL:create_file:notes.txt: Your ideas go here

User: List all files
AI: I'll show you all files. TOOL:list_files:

User: Set a reminder to buy groceries
AI: I'll set that reminder. TOOL:set_reminder:buy groceries

Current request: {user_message}
Response:"""

    print(f"👤 User: {user_message}")

    # Get AI response
    ai_response = ask_ai(system_prompt)
    print(f"🤖 Agent: {ai_response}")

    # Handle action calls
    if "TOOL:" in ai_response:
        tool_match = re.search(r'TOOL:(\w+):(.+)', ai_response)
        if tool_match:
            tool_name = tool_match.group(1)
            tool_input = tool_match.group(2).strip()

            print(f"🎬 Taking action: {tool_name}")

            if tool_name in ACTION_TOOLS:
                # Actually perform the action!
                action_result = ACTION_TOOLS[tool_name](tool_input)
                print(f"✅ Action result: {action_result}")

                # Log the action
                memory.log_action(f"{tool_name}({tool_input})", action_result)

                # Get final response
                final_prompt = system_prompt + ai_response + f"\nAction completed: {action_result}\nFinal response:"
                final_response = ask_ai(final_prompt)

                memory.add_conversation(user_message, final_response)
                return f"{ai_response}\nAction completed: {action_result}\n{final_response}"
            else:
                return f"{ai_response}\n❌ Unknown action: {tool_name}"

    # Regular conversation
    memory.add_conversation(user_message, ai_response)
    return ai_response

# Step 5: Test real actions
print("\n🧪 Testing real actions:")

test_actions = [
    "Create a file called 'my_goals.txt' with the content: Learn AI, Build agents, Change the world",
    "List all files in the current directory",
    "Read the my_goals.txt file back to me"
]

for action in test_actions:
    print(f"\n--- Action Test ---")
    response = action_agent(action)
    print(f"Final result: {response}")
    print("-" * 50)

# Step 6: Show what happened
print("\n🎬 Actions completed:")
for action in memory.completed_actions:
    print(f"  [{action['timestamp']}] {action['action']}")
    print(f"      → {action['result'][:60]}...")

# Check if files actually exist
print("\n📁 Files that actually exist now:")
try:
    files = [f for f in os.listdir('.') if f.endswith('.txt')]
    for file in files:
        print(f"  • {file}")
except:
    print("  Error listing files")

# Step 7: Interactive mode
def action_chat():
    """Interactive chat with action-taking agent"""
    print("\n🎬 Chat with your action-taking agent!")
    print("Try commands like:")
    print("  • 'Create a file called...'")
    print("  • 'List all files'")
    print("  • 'Set a reminder to...'")
    print("Type 'actions' to see what I've done")
    print("Type 'quit' to exit")

    while True:
        user_input = input("\n👤 You: ")

        if user_input.lower() == 'quit':
            print("👋 Goodbye!")
            break
        elif user_input.lower() == 'actions':
            print("\n🎬 Actions I've taken:")
            for action in memory.completed_actions:
                print(f"  {action['action']}")
            continue

        if user_input.strip():
            response = action_agent(user_input)
            print(f"🤖 Final: {response}")

# Ready to take actions? Uncomment the line below:
# action_chat()

print("\n🎉 Your agent can now take real actions!")
print("🚀 Course complete! You've built a full AI agent!")

Loading AI...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

✅ AI ready!
✅ Memory system ready
✅ Action tools ready: create_file, read_file, list_files, set_reminder, send_email

🧪 Testing real actions:

--- Action Test ---
👤 User: Create a file called 'my_goals.txt' with the content: Learn AI, Build agents, Change the world
🤖 Agent: I'll create the file 'my_goals.txt' with the content: Learn AI, Build agents, Change the world. TOOL:create_file:my_goals.txt: Learn AI, Build agents, Change the world
🎬 Taking action: create_file
✅ Action result: ✅ Created file: my_goals.txt
Final result: I'll create the file 'my_goals.txt' with the content: Learn AI, Build agents, Change the world. TOOL:create_file:my_goals.txt: Learn AI, Build agents, Change the world
Action completed: ✅ Created file: my_goals.txt
✅ Created file: my_goals.txt
--------------------------------------------------

--- Action Test ---
👤 User: List all files in the current directory
🤖 Agent: AI: I'll list all files in the current directory. TOOL:list_files:
Final result: AI: I'll list 